In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
path = '/content/drive/MyDrive/2023.10.27_미프5차_캐글대회'

train = pd.read_csv(os.path.join(path+'/train.csv'))
test = pd.read_csv(os.path.join(path + '/test.csv'))

In [ ]:
train

In [ ]:
train.drop(['Unnamed: 0'], axis=1, inplace=True)
train['timestamp'] = pd.to_datetime(train['timestamp'])
test.drop(['Unnamed: 0'], axis=1, inplace=True)
test['timestamp'] = pd.to_datetime(test['timestamp'])

train['year'] = train['timestamp'].dt.year
train['month'] = train['timestamp'].dt.month
train['day'] = train['timestamp'].dt.day
train['hour'] = train['timestamp'].dt.hour
train['minute'] = train['timestamp'].dt.minute
train['second'] = train['timestamp'].dt.second
train['microsecond'] = train['timestamp'].dt.microsecond

test['year'] = test['timestamp'].dt.year
test['month'] = test['timestamp'].dt.month
test['day'] = test['timestamp'].dt.day
test['hour'] = test['timestamp'].dt.hour
test['minute'] = test['timestamp'].dt.minute
test['second'] = test['timestamp'].dt.second
test['microsecond'] = test['timestamp'].dt.microsecond

In [ ]:
train['label'].value_counts()

In [ ]:
train_notna = train.dropna()

In [ ]:
train_notna.describe()

In [ ]:
target = 'label'
x = train_notna.drop([target, 'timestamp'], axis=1)
y = train_notna[target]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

### 랜덤 포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train, y_train)

y_pred = model.predict(x_val)
print(accuracy_score(y_pred, y_val))
print(f1_score(y_val, y_pred, average='macro'))
print(classification_report(y_val, y_pred))

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_pred_dt = model.predict(x_val)

# 평가하기
print("Accuracy : " , accuracy_score(y_val, y_pred_dt))
print('')
print("f1 score : " , f1_score(y_val, y_pred_dt, average='macro'))
print('')
print("classification report : " , classification_report(y_val, y_pred_dt))

### ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier(random_state=42)
model.fit(x_train, y_train)
y_pred_ex = model.predict(x_val)

# 평가하기
print("Accuracy : " , accuracy_score(y_val, y_pred_ex))
print('')
print("f1 score : " , f1_score(y_val, y_pred_ex, average='macro'))
print('')
print("classification report : " , classification_report(y_val, y_pred_ex))

In [ ]:
'''from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(x_train, y_train)
y_pred_gb = model.predict(x_val)

# 평가하기
print("Accuracy : " , accuracy_score(y_val, y_pred_gb))
print('')
print("f1 score : " , f1_score(y_val, y_pred_gb, average='macro'))
print('')
print("classification report : " , classification_report(y_val, y_pred_gb))'''

### HistGradientBoostingClassifier

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model = HistGradientBoostingClassifier(random_state=42)
model.fit(x_train, y_train)
y_pred_hgb = model.predict(x_val)

# 평가하기
print("Accuracy : " , accuracy_score(y_val, y_pred_hgb))
print('')
print("f1 score : " , f1_score(y_val, y_pred_hgb, average='macro'))
print('')
print("classification report : " , classification_report(y_val, y_pred_hgb))

### XGBClassifier

In [ ]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(random_state=42)
y_t_maped = y_train.map({1:1,2:2,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:10,11:0})
y_v_maped = y_val.map({1:1,2:2,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:10,11:0})
model_xgb.fit(x_train, y_t_maped)
y_pred_xgb = model_xgb.predict(x_val)

print("Accuracy : " , accuracy_score(y_v_maped, y_pred_xgb))
print('')
print("f1 score : " , f1_score(y_v_maped, y_pred_xgb, average='macro'))
print('')
print("classification report : " , classification_report(y_v_maped, y_pred_xgb))

### 최종적으로 예측

In [ ]:
test.head()

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model = HistGradientBoostingClassifier(random_state=42)
model.fit(x, y)
y_pred = model.predict(test.drop('timestamp', axis=1))

In [ ]:
y_pred = pd.DataFrame(y_pred)

In [ ]:
y_pred.to_csv('y_pred.csv')

In [ ]:
## 캐글에서 정확도 0.99282 나온 데이터

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
s_x, s_y = smote.fit_resample(x_train, y_train)

model.fit(s_x, s_y)
y_s_pred = model.predict(test.drop('timestamp', axis=1))

y_s_pred = pd.DataFrame(y_s_pred)
y_s_pred.to_csv('y_s_pred.csv')

In [ ]:
# SMOTE 시키면 오히려 정확도가 0.98685로 떨어진다.

In [ ]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(random_state=1)
model.fit(x_train, y_train)
y_pred_lgb = model.predict(x_val)

# 평가하기
print("Accuracy : " , accuracy_score(y_val, y_pred_lgb))
print('')
print("f1 score : " , f1_score(y_val, y_pred_lgb, average='macro'))
print('')
print("classification report : " , classification_report(y_val, y_pred_lgb))

# Voting (ExtraTrees, HistGradient, XGBoost)

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
extra = ExtraTreesClassifier(random_state=42)
hgb = HistGradientBoostingClassifier(random_state=42)
xgb = XGBClassifier(random_state=42)

voting_estimators = [('extra', extra), ('hgb', hgb), ('xgb', xgb)]
voting = VotingClassifier(estimators = voting_estimators, voting='soft')

In [ ]:
y_map = y.map({1:1,2:2,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:10,11:0})
voting.fit(x, y_map)
y_pred_vot = voting.predict(test.drop('timestamp', axis=1))

In [ ]:
y_pred_vot = pd.Series(y_pred_vot).map({1:1,2:2,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:10,0:11})

In [ ]:
y_pred_vot.to_csv('y_pred_vot.csv')

In [ ]:
# Voting 방식 정확도 - 0.99267

# GridSearch

In [ ]:
params = {'learning_rate' : [0.1, 0.2, 0.3, 0.4], 'max_bins' : [128, 192, 255]}

In [ ]:
model = HistGradientBoostingClassifier(random_state=1)

In [ ]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(model, params, n_jobs=-1)
gs.fit(x, y)
print(gs.best_params_)
print(gs.cv_results_['mean_test_score'])

In [ ]:
params = {'learning_rate' : [0.15, 0.2, 0.25], 'max_bins' : [96, 128, 160]}
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(model, params, n_jobs=-1)
gs.fit(x, y)
print(gs.best_params_)
print(gs.cv_results_['mean_test_score'])

# RandomSearch

In [ ]:
from scipy.stats import uniform, randint

In [ ]:
params = {'learning_rate' : uniform(0.15, 0.21), 'max_bins' : randint(112, 136)}
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(model, params, n_iter=12, n_jobs=-1, random_state=42)
rs.fit(x, y)

In [ ]:
print(rs.best_params_)
print(rs.cv_results_['mean_test_score'])

In [ ]:
# Grid Search에서 나온 0.2, 128로 찾아가겠습니다.

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model = HistGradientBoostingClassifier(learning_rate=0.2, max_bins=128, random_state=1)
model.fit(x_train, y_train)
y_pred_hgb = model.predict(x_val)

# 평가하기
print("Accuracy : " , accuracy_score(y_val, y_pred_hgb))
print('')
print("f1 score : " , f1_score(y_val, y_pred_hgb, average='macro'))
print('')
print("classification report : " , classification_report(y_val, y_pred_hgb))

In [ ]:
model.fit(x, y)
y_pred_gs = model.predict(test.drop('timestamp', axis=1))

y_pred_gs = pd.DataFrame(y_pred_gs)

In [ ]:
y_pred_gs.to_csv('y_pred_gs.csv')

In [ ]:
# RandomSearch에서 나온 수로 넣으면 오히려 성능이 떨어집니다.

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
model_rs = HistGradientBoostingClassifier(learning_rate= 0.1543227438021185, max_bins= 113, random_state=1)
model_rs.fit(x_train, y_train)
y_pred_hgb_rs = model_rs.predict(x_val)

# 평가하기
print("Accuracy : " , accuracy_score(y_val, y_pred_hgb_rs))
print('')
print("f1 score : " , f1_score(y_val, y_pred_hgb_rs, average='macro'))
print('')
print("classification report : " , classification_report(y_val, y_pred_hgb_rs))